## Exploratory Data Analysis

### Import and basic info

**Output** - aisles, orders, products, departments, order_products_prior, order_products_train

In [1]:
import pandas as pd
import numpy as np
import itertools
import pickle
from collections import Counter
from itertools import combinations, groupby, chain
import warnings
warnings.filterwarnings('ignore')

## Prepare Data for Market Basket Analysis

In [2]:
# order_products_prior = pd.read_csv("Data/order_products__prior.csv")
# order_products_train = pd.read_csv("Data/order_products__train.csv")
# orders = pd.read_csv("Data/orders.csv")
aisles = pd.read_csv("archive (24)/aisles.csv")
# grouped_users = pickle.load(open("Pickle/grouped_users.p", "rb"))
products_desc = pickle.load(open("Pickle/products_desc.p", "rb"))
merged_orders = pickle.load(open("Pickle/merged_orders.p", "rb"))
# orders_test = orders[orders['eval_set']=='test']
clustered_users = pickle.load(open("Pickle/clustered_users.p", "rb"))

In [3]:
# I want to calculate the association rules by each cluster.  It will make the data smaller.
merged_orders = merged_orders.sort_values(['order_id','product_id'])
merged_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
6076,14,18194,prior,49,3,15,3.0,162,3,1,Organic Mini Homestyle Waffles,52,1,frozen breakfast,frozen
6077,14,18194,prior,49,3,15,3.0,2452,4,1,Naturals Chicken Nuggets,129,1,frozen appetizers sides,frozen
3599,46,10951,prior,3,0,18,18.0,3703,17,0,"Plates, 10-1/16 Inch",111,17,plates bowls cups flatware,household
3600,46,10951,prior,3,0,18,18.0,4447,16,0,Berries 'n Cherries Fruit Snacks,50,19,fruit vegetable snacks,snacks
3601,46,10951,prior,3,0,18,18.0,4778,4,1,Wheat Thins Reduced Fat Crackers,78,19,crackers,snacks


In [4]:
clustered_users.reset_index(inplace=True)

In [5]:
# Merging these to add the cluster to each order
clustered_orders = merged_orders.merge(clustered_users[['user_id','cluster']], on = 'user_id')

## Market Basket Analysis

In [6]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
'''Python Generators - A generator is a special type of function that returns an iterable sequence of items.
However, unlike regular functions that return all the values at once, a generator yields one value at a time. To get the
next value in the set, we must ask for it — either by explicitly calling the generator’s built-in “next” method, or
implicitly via a for loop.  This is a great property of generators because it means that we don’t have to store all of
the values in memory at once. We can load and process one value at a time, discard that value when we finished, and move
on to process the next value. This feature makes generators perfect for creating item pairs and counting their
frequency of co-occurrence.'''
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), 
                       left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), 
                       left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [7]:
def association_rules(order_item, min_support):

#     print("Starting number of order_item pairs: {:22d}".format(len(order_item)))
    print("Starting number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                          order_item.nunique(),len(order_item))) 

    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
#     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaning number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                          order_item.nunique(),len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
#     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaining number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                          order_item.nunique(),len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [8]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,20):
    i = clustered_orders[clustered_orders['cluster'] == x]
    i = i[['order_id', 'product_id']]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [9]:
# run each cluster series through association rule function and obtain association rules at item level

group_association_rules_dic = {}
for x in range(0,20):
    print('Cluster: ', x)
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Cluster:  0
Starting number of orders, items, order_item pairs: 2043, 1000, 3211
Items with support >= 0.0001:            1000
Remaning number of orders, items, order_item pairs: 2043, 1000, 3211
Remaining orders with 2+ items:         785
Remaining number of orders, items, order_item pairs: 785, 771, 1953
Item pairs:                            1656
Item pairs with support >= 0.0001:       1656

Cluster:  1
Starting number of orders, items, order_item pairs: 59, 106, 148
Items with support >= 0.0001:             106
Remaning number of orders, items, order_item pairs: 59, 106, 148
Remaining orders with 2+ items:          44
Remaining number of orders, items, order_item pairs: 44, 103, 133
Item pairs:                             164
Item pairs with support >= 0.0001:        164

Cluster:  2
Starting number of orders, items, order_item pairs: 20, 34, 42
Items with support >= 0.0001:              34
Remaning number of orders, items, order_item pairs: 20, 34, 42
Remaining orders with 2+ ite

In [10]:
# format item association rule dataframes and merge product names

for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
               .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
               .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                    'product_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confAtoB',
                                                                    'confBtoA','lift']]

In [11]:
pickle.dump(group_association_rules_dic, open("Pickle/group_association_rules_dic.p", "wb"))

In [12]:
# group_association_rules_dic = pickle.load(open("Pickle/group_association_rules_dic.p", "rb"))

In [13]:
# example of association rule dataframe

x = group_association_rules_dic[19]
x.sort_values('lift', ascending = False)

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
0,2469,3801,Italian Salad Blend,Classic Cheddar Potatoes Pierogie,1,1.0,1,1.0,1,1.0,1.0,1.0,1.0
1,2469,4030,Italian Salad Blend,Egg Roll Wraps,1,1.0,1,1.0,1,1.0,1.0,1.0,1.0
2,3801,4030,Classic Cheddar Potatoes Pierogie,Egg Roll Wraps,1,1.0,1,1.0,1,1.0,1.0,1.0,1.0


## Market Basket Analysis at Aisle level

In [14]:
# separate order data by cluster and transform into expected format of group association function at aisle level

cluster_aisle_orders = {}
for x in range(0,20):
    i = clustered_orders[clustered_orders['cluster'] == x]
    i = i[['order_id', 'product_id', 'aisle_id']]
    i = i.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[x] = i

In [15]:
# run association function at aisle level

group_aisle_association_rules_dic = {}
for x in range(0,20):
    print("Cluster: ", x)
    group_aisle_association_rules_dic[x] = association_rules(cluster_aisle_orders[x],.0001)

Cluster:  0
Starting number of orders, items, order_item pairs: 2043, 106, 3088
Items with support >= 0.0001:             106
Remaning number of orders, items, order_item pairs: 2043, 106, 3088
Remaining orders with 2+ items:         743
Remaining number of orders, items, order_item pairs: 743, 99, 1788
Item pairs:                             862
Item pairs with support >= 0.0001:        862

Cluster:  1
Starting number of orders, items, order_item pairs: 59, 46, 144
Items with support >= 0.0001:              46
Remaning number of orders, items, order_item pairs: 59, 46, 144
Remaining orders with 2+ items:          44
Remaining number of orders, items, order_item pairs: 44, 46, 129
Item pairs:                             120
Item pairs with support >= 0.0001:        120

Cluster:  2
Starting number of orders, items, order_item pairs: 20, 23, 42
Items with support >= 0.0001:              23
Remaning number of orders, items, order_item pairs: 20, 23, 42
Remaining orders with 2+ items:   

In [16]:
pickle.dump(group_aisle_association_rules_dic, open("Pickle/group_aisle_association_rules_dic.p", "wb"))

In [25]:
group_aisle_association_rules_dic = pickle.load(open("Pickle/group_aisle_association_rules_dic.p", "rb"))

In [18]:
# reformat cluster aisle assocation dataframes

for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles
               .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles
               .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                    'aisle_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confAtoB',
                                                                    'confBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})

In [19]:
x = group_aisle_association_rules_dic[1].sort_values('lift',ascending = False)
x.sort_values('lift',ascending = False)

,aisle_A,aisle_B,aisle_name_A,aisle_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
0,16,26,fresh herbs,coffee,1,0.022727,1,0.022727,1,0.022727,1.000000,1.000000,44.000000
3,42,87,frozen vegan vegetarian,more household,1,0.022727,1,0.022727,1,0.022727,1.000000,1.000000,44.000000
2,81,87,canned jarred vegetables,more household,1,0.022727,1,0.022727,1,0.022727,1.000000,1.000000,44.000000
4,42,81,frozen vegan vegetarian,canned jarred vegetables,1,0.022727,1,0.022727,1,0.022727,1.000000,1.000000,44.000000
1,9,119,pasta sauce,frozen dessert,1,0.022727,1,0.022727,1,0.022727,1.000000,1.000000,44.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,45,95,candy chocolate,canned meat seafood,1,0.022727,34,0.772727,2,0.045455,0.029412,0.500000,0.647059
116,45,120,candy chocolate,yogurt,2,0.045455,34,0.772727,4,0.090909,0.058824,0.500000,0.647059
117,45,62,candy chocolate,white wines,1,0.022727,34,0.772727,2,0.045455,0.029412,0.500000,0.647059
118,19,45,oils vinegars,candy chocolate,1,0.022727,3,0.068182,34,0.772727,0.333333,0.029412,0.431373


## Recommender

In [20]:
group_association_rules_dic[0]

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
0,294,4739,Minis Original Saltine Crackers,Whole Chipotle Chili,1,0.001274,1,0.001274,1,0.001274,1.000000,1.000000,785.000000
1,2552,3305,Organic Raw Pomegranate Kombucha,Turkey Cranberr Sage Bone Broth,1,0.001274,1,0.001274,1,0.001274,1.000000,1.000000,785.000000
2,2604,5140,Lemon Zest Sorbetto,Lean & Fit Coconut Lemongrass Chicken,1,0.001274,1,0.001274,1,0.001274,1.000000,1.000000,785.000000
3,809,2950,Bolani Pumpkin Filled Flatbread,All Natural Zaatar Seasoning,1,0.001274,1,0.001274,1,0.001274,1.000000,1.000000,785.000000
4,2540,5103,Organic Juliette Vanilla Ice Cream Sandwiches,Blueberry & Purple Sweet Potato Super Puffs,1,0.001274,1,0.001274,1,0.001274,1.000000,1.000000,785.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,2295,5025,Yellow Bell Pepper,Green Onions,1,0.001274,31,0.039490,34,0.043312,0.032258,0.029412,0.744782
1652,4920,5479,Seedless Red Grapes,Italian Sparkling Mineral Water,1,0.001274,48,0.061146,23,0.029299,0.020833,0.043478,0.711051
1653,4605,4799,Yellow Onions,Shredded Parmesan,1,0.001274,58,0.073885,20,0.025478,0.017241,0.050000,0.676724
1654,4605,5479,Yellow Onions,Italian Sparkling Mineral Water,1,0.001274,58,0.073885,23,0.029299,0.017241,0.043478,0.588456


In [21]:
products_desc.set_index('product_id', inplace=True)

In [22]:
# returns top x items associated based on lift

def product_to_product(cluster, product_id, item_lift, product_name, num_products):
    df = group_association_rules_dic[cluster]
    df = df[(df['item_A'] == product_id) | (df['item_B'] == product_id)][['product_name_A','item_A','product_name_B','item_B','confAtoB','lift']].sort_values('lift', ascending = False)
    df = df[df['lift'] > item_lift]
    df = df.sort_values('lift', ascending = False)
    df = df.head(n = num_products)
    product_to_product_associations = df['product_name_A'].values.tolist()
    for x in df['product_name_B'].values.tolist():
        product_to_product_associations.append(x)
    product_to_product_associations = [x for x in product_to_product_associations if x != product_name]
    return product_to_product_associations

In [23]:
# returns recommended products given inputs

def pdp_recommender(user_id, product_id, item_lift, num_products):
    product_name = products_desc.at[product_id,'product_name']
    aisle_id = products_desc.at[product_id,'aisle_id']
    aisle_name = products_desc.at[product_id,'aisle']
    cluster = clustered_users.at[user_id, 'cluster']
    return product_to_product(cluster = cluster, product_id = product_id, item_lift = item_lift, 
                              product_name = product_name, num_products = num_products)

In [24]:
# results for users in the 20 different clusters on prodcut 39055 Mild Roja Salsa, lift >1, 5 products

for i in range(0,20):
    user = clustered_users[clustered_users['cluster']==i].sample().index[0]
    print('\ncluster', i, ': user ', user)
    print('\n'.join([x for x in (pdp_recommender(user, 3187, 1, 5))]))


cluster 0 : user  596


cluster 1 : user  12


cluster 2 : user  3096


cluster 3 : user  3332


cluster 4 : user  1275


cluster 5 : user  2823


cluster 6 : user  2116


cluster 7 : user  2696


cluster 8 : user  538


cluster 9 : user  3297


cluster 10 : user  2783


cluster 11 : user  48


cluster 12 : user  1344


cluster 13 : user  875


cluster 14 : user  1048


cluster 15 : user  3023


cluster 16 : user  954


cluster 17 : user  1984


cluster 18 : user  2415


cluster 19 : user  1356

